<a href="https://colab.research.google.com/github/VaggelisApostolou/auth-dws-ml-2025/blob/main/ML_Exercise9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [1]:
!pip install swig
!pip install gymnasium[box2d]
!pip install renderlab
!pip install stable-baselines3 shimmy moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 6.2 MB/s eta 0:00:00


# **Basic Functions**

* reset(): Η συνάρτηση αυτή επαναφέρει το περιβάλλον στην αρχική του κατάσταση.
* step(action): Αυτή η συνάρτηση εκτελεί το βήμα προσομοίωσης. Δέχεται ως είσοδο μια ενέργεια από τον πράκτορα και επιστρέφει 5 τιμές:
1. observation: Η νέα κατάσταση του περιβάλλοντος.
2. reward: Η ανταμοιβή που κερδήθηκε από την ενέργεια.
3. terminated: True αν ο πράκτορας τερμάτισε (π.χ. νίκησε ή έχασε/συνετρίβη)
4. truncated: True αν το επεισόδιο διακόπηκε (π.χ. όριο χρόνου).
5. info: Βοηθητικές πληροφορίες.
* render(): Χρησιμοποιείται για την οπτικοποίηση του περιβάλλοντος (γραφικά), ώστε να βλέπουμε τι κάνει ο πράκτορας.







# **LunarLander**

* Observation Space: Αποτελείται από ένα διάνυσμα 8 αριθμών που περιγράφουν: τις συντεταγμένες (x, y), τις ταχύτητες (x, y), την γωνία, την γωνιακή ταχύτητα και δύο boolean τιμές που δείχνουν αν τα πόδια του σκάφους ακουμπούν στο έδαφος.
* Action Space: Είναι διακριτός με 4 πιθανές ενέργειες:
  * 0: Μην κάνεις τίποτα.
  * 1: Πυροδότηση αριστερού κινητήρα προσανατολισμού.
  * 2: Πυροδότηση κύριου κινητήρα.
  * 3: Πυροδότηση δεξιού κινητήρα προσανατολισμού
* Reward Function: Η ανταμοιβή αυξάνεται καθώς το σκάφος πλησιάζει στο σημείο προσγείωσης και μειώνεται αν απομακρύνεται ή κινείται γρήγορα. Παίρνει έξτρα πόντους για κάθε πόδι που ακουμπάει στο έδαφος. Η συντριβή δίνει -100 πόντους, ενώ η ασφαλής προσγείωση +100. Η χρήση κινητήρων κοστίζει μικρή ποσότητα πόντων (καύσιμα).





# **Agent & Visualization**

In [2]:
import gymnasium as gym
import renderlab as rl
import numpy as np
env = gym.make("LunarLander-v3", render_mode="rgb_array")
env = rl.RenderFrame(env, "./output_random")
total_rewards = []
num_episodes = 5
for episode in range(num_episodes):
    observation, info = env.reset()
    episode_reward = 0
    while True:
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        if terminated or truncated:
            break
    total_rewards.append(episode_reward)
    print(f"Episode {episode+1}: Score = {episode_reward}")
env.play()
print(f"Μέσο Score Random Agent: {np.mean(total_rewards)}")

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



Episode 1: Score = -142.27248395846087
Episode 2: Score = -99.08500878983205
Episode 3: Score = -117.08981235257224
Episode 4: Score = -103.38613029930333
Episode 5: Score = -114.05884169175312
Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


Μέσο Score Random Agent: -115.17845541838433


# **Training with DQN & PPO**

In [3]:
from stable_baselines3 import DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
env_dqn = gym.make("LunarLander-v3", render_mode=None)
env_ppo = gym.make("LunarLander-v3", render_mode=None)
train_steps = 50000
model_dqn = DQN("MlpPolicy", env_dqn, verbose=1)
model_dqn.learn(total_timesteps=train_steps)
model_dqn.save("dqn_lunar")
model_ppo = PPO("MlpPolicy", env_ppo, verbose=1)
model_ppo.learn(total_timesteps=train_steps)
model_ppo.save("ppo_lunar")

  return datetime.utcnow().replace(tzinfo=utc)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


  return datetime.utcnow().replace(tzinfo=utc)



----------------------------------
| rollout/            |          |
|    ep_len_mean      | 107      |
|    ep_rew_mean      | -135     |
|    exploration_rate | 0.918    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 378      |
|    time_elapsed     | 1        |
|    total_timesteps  | 429      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.35     |
|    n_updates        | 82       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 95.2     |
|    ep_rew_mean      | -155     |
|    exploration_rate | 0.855    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 540      |
|    time_elapsed     | 1        |
|    total_timesteps  | 762      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.884    |
|    n_updates      

  warnings.warn(



Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 84.4     |
|    ep_rew_mean     | -163     |
| time/              |          |
|    fps             | 576      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 90.2        |
|    ep_rew_mean          | -173        |
| time/                   |             |
|    fps                  | 512         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008679241 |
|    clip_fraction        | 0.0481      |
|    clip_range           | 0.2         |
|    entropy_loss  

# **Comparison & Evaluation**

In [4]:
from stable_baselines3.common.monitor import Monitor

def evaluate_and_record(model, name, directory):
    eval_env = gym.make("LunarLander-v3", render_mode=None)
    eval_env = Monitor(eval_env)
    mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5, deterministic=True)
    print(f"\nAlgorithm: {name}")
    print(f"Mean Reward (5 episodes): {mean_reward:.2f} +/- {std_reward:.2f}")
    eval_env.close()
    print(f"Recording video for {name}...")
    video_env = gym.make("LunarLander-v3", render_mode="rgb_array")
    video_env = rl.RenderFrame(video_env, directory)
    observation, info = video_env.reset()
    while True:
        action, _ = model.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = video_env.step(action)
        if terminated or truncated:
            break

    video_env.close()
    video_env.play()
evaluate_and_record(model_dqn, "DQN", "./output_dqn")
evaluate_and_record(model_ppo, "PPO", "./output_ppo")


Algorithm: DQN
Mean Reward (5 episodes): -162.19 +/- 66.89
Recording video for DQN...
Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



t:  89%|████████▉ | 235/264 [00:00<00:00, 238.29it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file ./output_dqn/1768432902.180958.mp4, 720000 bytes wanted but 0 bytes read,at frame 263/264, at time 8.77/8.77 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4



Algorithm: PPO
Mean Reward (5 episodes): -275.08 +/- 11.60
Recording video for PPO...
Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



t:  99%|█████████▉| 990/1002 [00:03<00:00, 286.49it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file ./output_ppo/1768432908.7676442.mp4, 720000 bytes wanted but 0 bytes read,at frame 1001/1002, at time 33.37/33.37 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4


# **Hyperparameter Tuning**

In [5]:
model_opt_ppo = PPO(
    "MlpPolicy",
    env_ppo,
    learning_rate=0.0003,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1
)

print("Εκπαίδευση Optimized PPO...")
model_opt_ppo.learn(total_timesteps=100000) # Περισσότερα βήματα
evaluate_and_record(model_opt_ppo, "Optimized PPO", "./output_opt_ppo")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Εκπαίδευση Optimized PPO...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 85       |
|    ep_rew_mean     | -207     |
| time/              |          |
|    fps             | 728      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 1024     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 87.9         |
|    ep_rew_mean          | -171         |
| time/                   |              |
|    fps                  | 658          |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0015341276 |
|    clip_fraction        | 0            |
|    clip_range      

Moviepy - Done !
Moviepy - video ready temp-{start}.mp4
